In [1]:
import pandas as pd
import numpy as np

In [2]:
basedf = pd.read_excel('dados_vendas.xlsx')

In [3]:
basedf.sort_values(['HORA'],ascending=[1])
percentage=0.8
cutpoint=round(len(basedf)*percentage)
train,test=basedf[:cutpoint],basedf[cutpoint:]
print(len(train),len(test))

257393 64348


In [4]:
df= train[['COD_CLIENTE','COD_PRODUTO']]
df=df.drop_duplicates(keep='first')
v=df['COD_CLIENTE'].nunique()
u=df['COD_PRODUTO'].nunique()
convert_cliente={}
revert_cliente=['' for x in range(v)]
c=0
for x in df['COD_CLIENTE'].unique():
    convert_cliente[x]=c
    revert_cliente[c]=x
    c+=1
convert_produto={}
revert_produto=['' for x in range(u)]
c=0
for x in df['COD_PRODUTO'].unique():
    convert_produto[x]=c
    revert_produto[c]=x
    c+=1
lista_adj=[[] for x in range(v)]
lista_adj_u=[[] for x in range(u)]
for i,j in df.iterrows():
    lista_adj[convert_cliente[j[0]]].append(convert_produto[j[1]])
    lista_adj_u[convert_produto[j[1]]].append(convert_cliente[j[0]])
outputname="Adjacency_list.txt"
text_file = open(outputname, "w")
text_file.write(str(v)+'\n')
text_file.write(str(u)+'\n')
for j in lista_adj:
    txt=str(j)[1:-1].replace(',',' ')
    text_file.write( txt+'\n')
text_file.close()


In [5]:
import shlex
from subprocess import Popen, PIPE
def get_exitcode_stdout_stderr(cmd):
    """
    Execute the external command and get its exitcode, stdout and stderr.
    """
    args = shlex.split(cmd)

    proc = Popen(args, stdout=PIPE, stderr=PIPE)
    out, err = proc.communicate()
    exitcode = proc.returncode
    #
    return exitcode, out, err

cmd = './AMRPB Adjacency_list.txt'  # arbitrary external command, e.g. "python mytest.py"
exitcode, out, err = get_exitcode_stdout_stderr(cmd)

In [6]:

f = out.decode("utf-8").split('\n')[-1]
nonnumbers=['(',')','}',']']
text=f.split('}')
clustertext=text[0].split('[')
solotext=text[1].split('(')
#print(len(solotext[2]))
biclusters=[]
v_cluster=[[] for x in range(v)]
u_cluster=[[] for x in range(u)]
#Clusters
for i,a in enumerate(clustertext[1:]):
    bicluster=[]
    for j,b in enumerate(a.split('),')):
        array=[]
        for k,c in enumerate(b.replace('],','').split(',')):
            txt=c
            for r in nonnumbers:
                txt=txt.replace(r, "")
            if(txt!=''):
                code=int(txt)
                array.append(code)
                if(not j):
                    v_cluster[code].append(i)
                else:
                    u_cluster[code].append(i)
        bicluster.append(array)
    biclusters.append(bicluster)
#Solos
#for i,a in enumerate(solotext[1:]):
#    for j,b in enumerate(a.split(',')):
#        txt=b
#        for r in nonnumbers:
#            txt=txt.replace(r, "")
#       if(txt!=''):
#           code=int(txt)
#            new_row = {'code':code, 'cluster':None }
#            if(j):
#               v_cluster[code].append()
#           else:
#                u_cluster[code].append()
print(u_cluster)

[[84, 89, 122], [66, 89, 105], [65, 76, 78, 93, 94, 102, 104, 110, 116, 122], [], [], [86, 105], [87, 97], [], [], [97, 129], [4, 31, 60], [89, 107], [], [89], [111, 132], [], [10, 11, 15, 43, 51, 74, 89, 93, 105, 107, 122, 130, 132], [], [], [15, 20, 45], [0, 11, 20, 58], [89], [], [], [66, 78], [], [85], [], [], [], [], [71, 107], [71, 97], [], [92], [], [], [], [], [98, 125], [124], [89], [], [], [], [], [], [66, 89, 108], [89], [], [97], [], [], [89], [12, 32, 61], [0, 12, 22, 44], [], [], [], [66, 102], [], [7, 27, 53], [66], [26, 62], [], [85], [106], [85, 87], [], [], [], [], [], [], [], [2, 29, 56], [], [12, 31, 61], [122], [], [15, 23, 43], [], [6, 33, 52], [85, 89], [], [66, 92, 115], [105], [], [], [], [8, 19, 46], [34, 56], [], [], [98], [9, 31, 60], [62], [9, 28, 52], [2, 25, 54], [], [13, 23, 49], [], [], [92], [], [98, 115], [], [], [], [1, 34, 61], [85], [], [7, 37, 51], [98], [41, 54], [15, 31, 57], [69], [87], [34], [], [], [], [], [], [], [], [], [39, 56], [8, 23, 46

In [7]:
def similarity (cod_viewed,cod_recomended,lista_adj_u,cluster_u):
    total=len(cluster_u)
    partial=len(set(cluster_u) & set(lista_adj_u[cod_recomended]))
    #print(total,partial)
    #print(set(recomendations) & set(lista_adj_u[cod_recomended]),sep='\n')
    if(partial==0):
        return 0
    return partial/total
        

In [8]:
#def similarity (cod_viewed,cod_recomended,lista_adj_u,recomendations):
#    total=len(recomendations)
#    clusters
#    partial=len(set(recomendations) & set(lista_adj_u[cod_recomended]))
#    print(total,partial)
#    print(set(recomendations) & set(lista_adj_u[cod_recomended]),sep='\n')
#    if(partial==0):
#        return 0
#    return partial/total
        

### Cliente

In [9]:
def recomenda_cliente(cliente,convert_cliente,v_cluster,biclusters):
    try:
        recomendations=[]
        cliente_key=convert_cliente[cliente]
        clusters=v_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        recomendations.append(x)
        return recomendations
    except KeyError:
        return 404 #"Cliente não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Produto

In [133]:
def recomenda_produto(produto,convert_produto,u_cluster,biclusters):
    try:
        recomendations=[]
        produto_key=convert_produto[produto]
        clusters=u_cluster[produto_key]
        scores=[]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        if(x not in recomendations and x!= produto_key):
                            recomendations.append(x)
                            scores.append(similarity(produto_key,x,lista_adj_u,b))
        return [recomendations,scores]
    except KeyError:
        return 404 #"Produto não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Usando vizinhos segundos para clientes

In [21]:
def recomenda_cliente_segundos(cliente,convert_cliente,v_cluster,biclusters):
    try:
        recomendations=[]
        cliente_key=convert_cliente[cliente]
        clusters=v_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==1):
                    for x in b:
                        recomendations.append(x)
                else:
                    for viz in b:
                        for clu in v_cluster[viz]:
                            for i,el in enumerate(biclusters[clu]):
                                if(i==1):
                                    for pro in el:
                                        if(pro not in recomendations):
                                            #print(pro)
                                            recomendations.append(pro)
        #comprados=[]
        #compras_cliente=newdf[newdf.COD_CLIENTE==cliente]
        #for x in compras_cliente.COD_PRODUTO:
        #    comprados.append(convert_produto[x])
        #recomendations=list(filter(lambda a: a not in comprados, recomendations))
        return recomendations
    except KeyError:
        return 404 #"Cliente não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


### Usando vizinhos segundos para produtos

In [12]:
def recomenda_produto_segundos(produto):
    try:
        recomendations=[]
        produto_key=convert_produto[produto]
        clusters=u_cluster[cliente_key]
        for a in clusters:
            for j,b in enumerate(biclusters[a]):
                if(j==0):
                    for x in b:
                        recomendations.append(x)
                else:
                    for viz in b:
                        for clu in v_cluster[viz]:
                            for i,el in enumerate(biclusters[clu]):
                                if(i==1):
                                    for pro in el:
                                        if(pro not in recomendations):
                                            print(pro)
                                            recomendations.append(pro)
        print(recomendations)
    except KeyError:
        return 404 #"Produto não encontrado"
    except:
        return -1 #"Erro inesperado encontrado, contacte o administrador"


In [121]:
test= test[['COD_CLIENTE','COD_PRODUTO']]
test=test.drop_duplicates(keep='first')
v=test['COD_CLIENTE'].nunique()
u=test['COD_PRODUTO'].nunique()
#convert_cliente
#revert_cliente
#convert_produto
#revert_produto
print(test.shape)
test_client=test[test['COD_CLIENTE'].isin(convert_cliente)]
test_client.reset_index(drop=True,inplace=True)
test_client=test_client.astype('str')
test_client=test_client.groupby(by='COD_CLIENTE',as_index=False).agg(','.join)

test_product=test[test['COD_PRODUTO'].isin(convert_produto)]
test_product.reset_index(drop=True,inplace=True)
test_product=test_product.astype('str')
test_product=test_product.groupby(by='COD_PRODUTO',as_index=False).agg(','.join)

(62973, 2)


### Por Cliente

In [122]:
got_recomended=0
right_recomendations=0
for i,x in test_client.iterrows():
    total=len(x[1].split(','))
    compras=x[1].split(',')
    recomendations=recomenda_cliente_segundos(x[0],convert_cliente,v_cluster,biclusters)
    recomendations=[revert_produto[x] for x in recomendations]
    if(recomendations!=[]):
        got_recomended+=1
    recomended=np.intersect1d(compras,recomendations)
    if(recomended!=[]):
        right_recomendations+=len(recomended)
print("Clientes na base de teste","\t|Clientes após filtros","\t|Clientes que receberam recomendações","\t|Produtos Recomendados Corretamente")
print(test['COD_CLIENTE'].nunique(),"\t\t\t\t|",len(test_client),"\t\t|",got_recomended,"\t\t\t\t\t|",right_recomendations)

<ipython-input-122-72f87089285d>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(recomended!=[]):


Clientes na base de teste 	|Clientes após filtros 	|Clientes que receberam recomendações 	|Produtos Recomendados Corretamente
54681 				| 13152 		| 534 					| 99


### Por Cliente do Produto 

In [ ]:

right_recomendations=0
cliente_search=test.reset_index(drop=True)
cliente_search=cliente_search.astype('str')
cliente_search=cliente_search.groupby(by='COD_CLIENTE',as_index=False).agg(','.join)
cliente_search=cliente_search.set_index('COD_CLIENTE')
for i,x in test_product.iterrows():
    compras=[]
    total=len(x[1].split(','))
    compradores=x[1].split(',')
    for y in compradores:
        for z in cliente_search.loc[y].values:
            for w in z.split(','):
                if(w not in compras and w!=x[0]):
                    compras.append(w)
    recomendations=recomenda_produto(int(x[0]),convert_produto,u_cluster,biclusters)[0]
    recomendations=[revert_produto[x] for x in recomendations]
    recomended=np.intersect1d(compras,recomendations)
    if(recomended!=[]):
        right_recomendations+=len(recomended)
    #print(x[0],compras,recomendations)
print("Produtos na base de teste","\t|Produtos após filtros","\t|Produtos Recomendados Corretamente")
print(test['COD_PRODUTO'].nunique(),"\t\t\t\t|",len(test_product),"\t\t\t|",right_recomendations)

<ipython-input-158-b80c2ce841ac>:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(recomended!=[]):
<ipython-input-158-b80c2ce841ac>:18: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if(recomended!=[]):
